In [2]:
# Import libraries

import pandas as pd 
import pycaret as pyc

In [3]:
# Load the data

data = pd.read_csv("data/Customer-Churn.csv")
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
target = 'Churn'
type = 'classification'

In [5]:
data[target].value_counts(normalize=True)

Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

In [6]:
# Initialize the setup
from pycaret.classification import *
clf1 = setup(data, target = target, session_id = 123,
             normalize = True,
             transformation = True,
             feature_selection=True,
            #  ignore_low_variance = True,
             remove_multicollinearity = True,
             multicollinearity_threshold = 0.95,
             fix_imbalance = True,
            #  fix_imbalance_method = 'SMOTE',
             log_experiment = "mlflow",
             verbose = False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3622, number of negative: 3622
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6617
[LightGBM] [Info] Number of data points in the train set: 7244, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

In [ ]:
# Get preproccessed data


In [7]:
# Compare Models
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.7357,0.5330,0.7357,0.5657,0.6359,0.0438,0.0460,0.1380
lr,Logistic Regression,0.7347,0.8097,0.7347,0.5398,0.6223,0.0000,0.0000,0.8380
knn,K Neighbors Classifier,0.7347,0.5000,0.7347,0.5398,0.6223,0.0000,0.0000,0.8280
nb,Naive Bayes,0.7347,0.5000,0.7347,0.5398,0.6223,0.0000,0.0000,0.1390
dt,Decision Tree Classifier,0.7347,0.5000,0.7347,0.5398,0.6223,0.0000,0.0000,0.1380
svm,SVM - Linear Kernel,0.7347,0.4933,0.7347,0.5398,0.6223,0.0000,0.0000,0.1470
ridge,Ridge Classifier,0.7347,0.8097,0.7347,0.5398,0.6223,0.0000,0.0000,0.1380
rf,Random Forest Classifier,0.7347,0.5234,0.7347,0.5398,0.6223,0.0000,0.0000,0.1720
ada,Ada Boost Classifier,0.7347,0.5000,0.7347,0.5398,0.6223,0.0000,0.0000,0.1400
gbc,Gradient Boosting Classifier,0.7347,0.4999,0.7347,0.5398,0.6223,0.0000,0.0000,0.1930


2024/08/12 08:21:37 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 08:21:37 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 08:21:37 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 08:21:38 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 08:21:38 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/12 08:21:39 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when

In [13]:
# Finetune best model
from pycaret.classification import tune_model

tuned_best, tuner = tune_model(best, return_tuner = True, search_library="optuna" )

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7363,0.8225,0.7363,0.5421,0.6245,0.0000,0.0000
1,0.7363,0.8138,0.7363,0.5421,0.6245,0.0000,0.0000
2,0.7343,0.7761,0.7343,0.5392,0.6218,0.0000,0.0000
3,0.7343,0.8135,0.7343,0.5392,0.6218,0.0000,0.0000
4,0.7343,0.8025,0.7343,0.5392,0.6218,0.0000,0.0000
5,0.7343,0.7895,0.7343,0.5392,0.6218,0.0000,0.0000
6,0.7343,0.8352,0.7343,0.5392,0.6218,0.0000,0.0000
7,0.7343,0.7997,0.7343,0.5392,0.6218,0.0000,0.0000
8,0.7343,0.8300,0.7343,0.5392,0.6218,0.0000,0.0000


[I 2024-08-12 09:35:15,425] Searching the best hyperparameters using 4930 samples...
[I 2024-08-12 09:37:43,771] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


2024/08/12 09:38:42 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [14]:
# Display the Tuner. 

tuner

OptunaSearchCV(callbacks=None,
               cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
               enable_pruning=False, error_score='raise',
               estimator=Pipeline(memory=Memory(location=None),
                                  steps=[('label_encoding',
                                          TransformerWrapperWithInverse(exclude=None,
                                                                        include=None,
                                                                        transformer=LabelEncoder())),
                                         ('numerical_imputer',
                                          TransformerWrapper(exclude=None,
                                                             incl...
               param_distributions={'actual_estimator__shrinkage': FloatDistribution(high=1.0, log=True, low=0.0001, step=None),
                                    'actual_estimator__solver': CategoricalDistribution(choices=('lsqr', 'eigen'))},
               random_state=123, refit=False, return_train_score=False,
               scoring='accuracy',
               study=<optuna.study.study.Study object at 0x0000028BB94621D0>,
               subsample=1.0, timeout=None, verbose=1)

In [9]:
# !mlflow ui

^C


In [15]:
final_model = create_model(tuned_best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7363,0.5000,0.7363,0.5421,0.6245,0.0000,0.0000
1,0.7363,0.5000,0.7363,0.5421,0.6245,0.0000,0.0000
2,0.7343,0.5000,0.7343,0.5392,0.6218,0.0000,0.0000
3,0.7343,0.5000,0.7343,0.5392,0.6218,0.0000,0.0000
4,0.7343,0.5000,0.7343,0.5392,0.6218,0.0000,0.0000
5,0.7343,0.5000,0.7343,0.5392,0.6218,0.0000,0.0000
6,0.7343,0.5000,0.7343,0.5392,0.6218,0.0000,0.0000
7,0.7343,0.5000,0.7343,0.5392,0.6218,0.0000,0.0000
8,0.7444,0.8301,0.7444,0.7980,0.7575,0.4381,0.4598


2024/08/12 09:39:07 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [16]:
# Dashboard 

from explainerdashboard import ClassifierExplainer, ExplainerDashboard

In [19]:
dashboard(final_model, display_format="inline")

Note: shap values for shap='kernel' normally get calculated against X_background, but paramater X_background=None, so setting X_background=shap.sample(X, 50)...
Generating self.shap_explainer = shap.KernelExplainer(model, X, link='identity')
Building ExplainerDashboard..
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...


  0%|          | 0/2113 [00:00<?, ?it/s]

Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating pred_percentiles...
Calculating predictions...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard inline (terminate it with ExplainerDashboard.terminate(8050))
